# Polymer Properties Machine Learning
Data and methodology taken from: Estimation and Prediction of the Polymers’ Physical Characteristics Using the Machine Learning Models Polymers 2024, 16(1), 115; https://doi.org/10.3390/polym16010115.

Github repository: https://github.com/catauggie/polymersML/tree/main

The goal of this notebook is to begin the machine learning phase which follows on from importing the polymer properties and analysisng the data set

In [1]:
# Go ahead and import the data into a dataframe and then make sure its imported properly by listing the first few lines
import pandas as pd
poly_data = pd.read_excel('resulting_dataset_master.xlsx')
poly_data.head(3)

,Unnamed: 0,polymer_name,Number of features,Features names,Flexural creep strain_value_median,Dielectric loss tangent_value_median,Tensile creep rupture time_value_median,Dynamic mechanical properties loss modulus_value_median,Pvt relation temperature_value_median,Vicat softening temperature_value_median,...,Thermal decomposition temperature_value_variance,Dynamic mechanical properties loss tangent_value_variance,Hansen parameter delta-d: dispersion component_value_variance,Dynamic viscosity loss tangent_value_variance,Volume resistivity_value_variance,Water absorption_value_variance,Heat of fusion_value_variance,Dynamic flexural properties storage modulus_value_variance,Dynamic shear properties storage modulus_value_variance,vector
0,0,polyethene,93,"['Density', 'Specific volume', 'Refractive ind...",NaN,0.00045,10.0,0.150,180.0,123.0,...,7598.312090,0.008877,0.0,NaN,3354844969133619982821830547268557602816,0.001996,0.000221,1.138410,3.445363e-01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,poly(prop-1-ene),86,"['Density', 'Specific volume', 'Refractive ind...",NaN,0.00100,20000.0,0.086,95.0,138.0,...,6815.698995,0.022569,NaN,588.0,1620381465120619970405709793344880640,0.610991,0.000071,100.427099,1.030412e+00,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,poly(but-1-ene),41,"['Density', 'Specific volume', 'Refractive ind...",NaN,NaN,NaN,0.140,240.9,99.0,...,NaN,NaN,NaN,NaN,,NaN,0.000152,NaN,1.159326e-09,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
import pandas as pd
import ast  # Module for literal_eval function
import numpy as np

df = poly_data

# Convert the string representations of lists to actual lists using ast.literal_eval
df['vector'] = df['vector'].apply(ast.literal_eval)

# Find the maximum length of lists in the 'vector' column
max_length = max(len(lst) for lst in df['vector'])

# Pad the lists with zeros to make them all the same length
padded_lists = [lst + [0] * (max_length - len(lst)) for lst in df['vector']]

# Create a new DataFrame with the padded lists
new_df = pd.DataFrame(padded_lists, columns=[f'col_{i}' for i in range(max_length)])

# Concatenate the new DataFrame with the original one
#df = pd.concat([df, new_df], axis=1)
new_df
# Display the new DataFrame

In [ ]:
new_df.to_csv('cols.csv')
new_df.to_excel('cols.xslx')